In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv
from rich.pretty import Pretty 
import os

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=os.getenv("OPEN_AI_API_KEY")
)

**Tool creation**

In [4]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [5]:
tools = [multiply]
tool_map = {tool.name:tool for tool in tools}

In [8]:
Pretty(tool_map)

{
    'multiply': StructuredTool(
        name='multiply',
        description='Multiply two numbers.',
        args_schema=<class 'langchain_core.utils.pydantic.multiply'>,
        func=<function multiply at 0x7d1856d2b920>
    )
}

**Binding Tools**

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
question = "3 multiplied by 2"

In [11]:
messages = [
    SystemMessage("You're a helpful assistant"),
    HumanMessage(question)
]

In [12]:
ai_message = llm_with_tools.invoke(messages)

In [14]:
Pretty(ai_message)

AIMessage(
    content='',
    additional_kwargs={
        'tool_calls': [
            {
                'id': 'call_BylW79aJYgdLPaN5gG3wdpNB',
                'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'},
                'type': 'function'
            }
        ],
        'refusal': None
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 54,
            'total_tokens': 71,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4o-mini-2024-07-18',
        'system_fingerprint': 'fp_560af6e559',
        'finish_reason': 'tool_calls',
        'logprobs': None
    },
    id='run--e86bb74d-8a6a-47fb-825b-4ac602585203-0',
    tool_calls=[
        {
            'name': 'multiply',
            'args': {'a': 3, 'b': 2},
            'id': 'call_BylW79aJYgdLPaN5gG3wdpNB',
            'type': 'tool_call'
        }
    ],
    usage_metadata={
        'input_tokens': 54,
        'output_tokens': 17,
        'total_tokens': 71,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

In [15]:
messages.append(ai_message)

In [16]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BylW79aJYgdLPaN5gG3wdpNB', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 54, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e86bb74d-8a6a-47fb-825b-4ac602585203-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_BylW79aJYgdLPaN5gG3wdpNB', 'type': 'tool_call'}], usage_

**Using Tool Calls**

In [17]:
parsed_tool_calls = parse_tool_calls(
    ai_message.additional_kwargs.get("tool_calls")
)

In [18]:
parsed_tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 2},
  'id': 'call_BylW79aJYgdLPaN5gG3wdpNB',
  'type': 'tool_call'}]

In [19]:
for tool_call in parsed_tool_calls:
    tool_call_id = tool_call['id']
    function_name = tool_call['name']
    arguments = tool_call['args']
    func = tool_map[function_name]
    result = func.invoke(arguments)
    tool_message = ToolMessage(
        content=result,
        name=function_name,
        tool_call_id=tool_call_id,
    )
    messages.append(tool_message)

**Sending the result back to the LLM**

In [20]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BylW79aJYgdLPaN5gG3wdpNB', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 54, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e86bb74d-8a6a-47fb-825b-4ac602585203-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_BylW79aJYgdLPaN5gG3wdpNB', 'type': 'tool_call'}], usage_

In [21]:
ai_message = llm_with_tools.invoke(messages)

In [22]:
ai_message

AIMessage(content='3 multiplied by 2 is 6.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 79, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--c077998c-1163-437e-9ee6-c225734f56ee-0', usage_metadata={'input_tokens': 79, 'output_tokens': 10, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})